In [1]:
from collections import deque

# Create the graph

We write the graph of pipes to a dict `conn` where `conn[node] = [adj_node1, adj_node2]` indicates that the pipe at `node` tries to connect to `adj_node1` and `adj_node1`. Two nodes `node1` and `node2` are actually connected if and only if `node1 in conn[node2]` and `node2 in conn[node1]`.

In [2]:
conn = {}
start = None

filepath = "./data/day10.txt"

with open(filepath) as f:
    row = 0
    while line := f.readline():
        line = line.rstrip()
        for col, c in enumerate(line):
            pos = (row, col)
            if c == 'S':
                start = pos
            elif c == '.':
                continue
            elif c == 'F':
                conn[pos] = [(row+1, col), (row, col+1)]
            elif c == '7':
                conn[pos] = [(row+1, col), (row, col-1)]
            elif c == 'J':
                conn[pos] = [(row-1, col), (row, col-1)]
            elif c == 'L':
                conn[pos] = [(row-1, col), (row, col+1)]
            elif c == '-':
                conn[pos] = [(row, col-1), (row, col+1)]
            elif c == '|':
                conn[pos] = [(row+1, col), (row-1, col)]
            else:
                print('we should not be here!', c)
        row += 1

In [3]:
start

(83, 25)

We don't need to figure out what the start node pipe actually is. We can simply set it to "connect" to all 4 possible cells.

In [4]:
conn[start] = [(start[0]+i, start[1]) for i in [-1, 1]] + [(start[0], start[1]+i) for i in [-1, 1]]
conn[start]

[(82, 25), (84, 25), (83, 24), (83, 26)]

# Part 1

BFS from start node. Keep track of the length of the path and print max length.

In [5]:
q = deque()
q.append(start + (0,))
visited = {start}
m = 0
while q:
    row, col, d = q.pop()
    m = d
    for row2, col2 in conn[(row, col)]:
        if (row2, col2) in visited:
            continue
        if (row2, col2) not in conn:
            continue
        if (row, col) not in conn[(row2, col2)]:
            continue
        q.appendleft((row2, col2, d+1))
        visited.add((row2, col2))
m

6754

# Part 2

Plan: We'll double the size of the grid. We will fill only loop nodes and the spaces between two connected loop nodes. Then we'll BFS from every non-loop node to see if we can get to the edge or not. If we can, it is an "outside" node; if not it is enclosed.

First, let's record which nodes are in the loop.

In [6]:
loop = {start}
q = deque()
q.append(start)
while q:
    node = q.pop()
    for adj_node in conn[node]:
        if adj_node in loop:
            continue
        if adj_node not in conn:
            continue
        if node not in conn[adj_node]:
            continue
        q.appendleft(adj_node)
        loop.add(adj_node)

In [7]:
len(loop)

13508

We need the number of rows and columns for bounds checking so let's save them in variables.

In [8]:
max_row, max_col = 140, 140

Now, let's figure out which cells or nodes are filled in the large grid. We first define a helper function to check if two nodes are connected:

In [9]:
def connected(node1, node2):
    if (node1 not in conn) or (node2 not in conn):
        return False
    return (node1 in conn[node2]) and (node2 in conn[node1])

Then we create the big loop, i.e. fill the cells in the large grid:

In [10]:
big_loop = set()
for row, col in loop:
    big_loop.add((2*row, 2*col))
    for i in [-1, 1]:
        if connected((row, col), (row+i, col)):
            big_loop.add((2*row+i, 2*col))
        if connected((row, col), (row, col+i)):
            big_loop.add((2*row, 2*col+i))

The size of the big loop should be exactly twice the length of the original loop:

In [11]:
len(big_loop), 2*len(loop)

(27016, 27016)

Now we are ready to compute the enclosed points. We'll run BFS starting at every non-loop point. If we get to the edge, we will mark all the points in the BFS as outside points. If we did not reach the edge, we mark the points as inside points, i.e. enclosed points.

Note that we loop points from the original grid, but the BFS is done in the doubled grid. So when we mark points as outside/inside after the BFS, we only consider points from the oroginal grid and we need to rescale the points back to the original grid, i.e. (row, col) -> (row//2, col//2).

In [12]:
outside_points = set()
inside_points = set()
for r in range(max_row):
    for c in range(max_col):
        point = (r, c)
        if point in loop:
            continue
        if point in outside_points:
            continue
        if point in inside_points:
            continue
        outside_flag = False
        path = set()
        path.add((2*r, 2*c))
        q = deque()
        q.append((2*r, 2*c))
        while q:
            row, col = q.pop()
            for i in [-1, 1]:
                if row+i < 0 or row+i >= 2*max_row:
                    outside_flag = True
                    continue
                if (row+i, col) in big_loop:
                    continue
                if (row+i, col) in path:
                    continue
                path.add((row+i, col))
                q.appendleft((row+i, col))
            for i in [-1, 1]:
                if col+i < 0 or col+i >= 2*max_col:
                    outside_flag = True
                    continue
                if (row, col+i) in big_loop:
                    continue
                if (row, col+i) in path:
                    continue
                path.add((row, col+i))
                q.appendleft((row, col+i))
        if outside_flag:
            for a, b in path:
                if a%2 == 0 and b%2 == 0:
                    outside_points.add((a//2, b//2))
        else:
            for a, b in path:
                if a%2 == 0 and b%2 == 0:
                    inside_points.add((a//2, b//2))

To make sure the numbers make sense, we print:
- number of points in the entire grid,
- number of points in the loop,
- number of points outside the loop,
- number of points inside the loop,
- total points - loop points - outside points (which needs to be equal to the number of points inside the loop).

In [13]:
max_row*max_col, len(loop), len(outside_points), len(inside_points), max_row*max_col-len(loop)-len(outside_points)

(19600, 13508, 5525, 567, 567)